## 4. AutoML-choice

### Dependencies

Installing dependencies (commented out, as they have already been intalled)

Help document can be found here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models

## Access datastore

In [1]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

DS_PATH = 'cer_auto_hezhang'

ws = Workspace.from_config()

### Reference labeling data set

In [5]:
training_dataset = Dataset.get_by_name(ws, name='cer_auto_hezhang')

In [6]:
training_dataset.to_pandas_dataframe().head()

,image_url,image_details,label
0,cer_auto_hezhang/train/img000001.jpg,None,None
1,cer_auto_hezhang/train/img000002.jpg,None,None
2,cer_auto_hezhang/train/img000003.jpg,None,None
3,cer_auto_hezhang/train/img000004.jpg,None,None
4,cer_auto_hezhang/train/img000005.jpg,None,None


## Now preparing the run

remember to look at https://github.com/swatig007/automlForImages/blob/main/ObjectDetection/AutoMLImage_ObjectDetection_SampleNotebook.ipynb

In [7]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "gpu-cluster-nc6"

try:
    compute_target = ws.compute_targets[cluster_name]
    print('Found existing compute target.')
except KeyError:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6', 
                                                           idle_seconds_before_scaledown=1800,
                                                           min_nodes=0, 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating a new compute target...
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
import azureml.core
print(azureml.core.VERSION)

1.37.0


In [9]:
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import GridParameterSampling, RandomParameterSampling, BayesianParameterSampling
from azureml.train.hyperdrive import BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, uniform
from azureml.automl.core.shared.constants import ImageTask

parameter_space = {
    'model': choice(
        {
            'model_name': choice('yolov5'),
            'learning_rate': uniform(0.0001, 0.01),
            #'model_size': choice('small', 'medium'), # model-specific
            'img_size': choice(640, 704, 768), # model-specific
        },
        {
            'model_name': choice('fasterrcnn_resnet50_fpn'),
            'learning_rate': uniform(0.0001, 0.001),
            #'warmup_cosine_lr_warmup_epochs': choice(0, 3),
            'optimizer': choice('sgd', 'adam', 'adamw'),
            'min_size': choice(600, 800), # model-specific
        }
    )
}

tuning_settings = {
    'iterations': 20, 
    'max_concurrent_iterations': 4, 
    'hyperparameter_sampling': RandomParameterSampling(parameter_space),  
#     'policy': BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=6),
}


automl_image_config = AutoMLImageConfig(task=ImageTask.IMAGE_OBJECT_DETECTION,
                                        compute_target=compute_target,
                                        training_data=training_dataset,
#                                         primary_metric='mean_average_precision',
                                        **tuning_settings)

In [10]:

from azureml.core import Experiment

experiment_name = 'automl-image-object-detection' 
experiment = Experiment(ws, name=experiment_name)

In [11]:
automl_image_run = experiment.submit(automl_image_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-image-object-detection,AutoML_26f6e1b3-0d0e-4a8d-8e82-a6a13e430733,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
automl_image_run.wait_for_completion(wait_post_processing=True)

{'runId': 'AutoML_26f6e1b3-0d0e-4a8d-8e82-a6a13e430733',
 'target': 'gpu-cluster-nc6',
 'status': 'Completed',
 'startTimeUtc': '2022-01-28T07:02:55.521439Z',
 'endTimeUtc': '2022-01-28T08:57:40.455691Z',
 'services': {},
 'properties': {'num_iterations': '1',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'mean_average_precision',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'gpu-cluster-nc6',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-image-object-detection","subscription_id":"7b60df94-2d4a-463d-9468-5ed1cce920b2","resource_group":"cer","workspace_name":"cer","region":"australiaeast","compute_target":"gpu-cluster-nc6","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1,"primary_metric"

In [ ]:
## register model by run id

In [ ]:
# best_child_run = ws.get_run('AutoML_a1f17867-2f0c-40e2-9753-068e663ca84e_HD_0')
best_child_run = automl_image_run.get_best_child()
model_name = best_child_run.properties['model_name']
model = best_child_run.register_model(model_name = model_name, model_path='outputs/model.pt')

## register model

In [ ]:
# best_child_run = automl_image_run.get_best_child()
# model_name = best_child_run.properties['model_name']
# model = best_child_run.register_model(model_name = model_name, model_path='outputs/model.pt')

### Run till here.............

## deploy model to AKS


In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.exceptions import ComputeTargetException

# Choose a name for your cluster
aks_name = "cluster-aks-gpu"

# Check to see if the cluster already exists
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    # Provision AKS cluster with GPU machine
    prov_config = AksCompute.provisioning_configuration(vm_size="STANDARD_NC6", 
                                                        location="australiaeast")
    # Create the cluster
    aks_target = ComputeTarget.create(workspace=ws, 
                                      name=aks_name, 
                                      provisioning_configuration=prov_config)
    aks_target.wait_for_completion(show_output=True)

## local deploy

In [ ]:
from azureml.core.model import InferenceConfig

best_child_run.download_file('outputs/scoring_file_v_1_0_0.py', output_file_path='score.py')
environment = best_child_run.get_environment()
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

## build local container

In [ ]:
from azureml.core.webservice import LocalWebservice
from azureml.core import Model

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=5002)

local_service = Model.deploy(ws, "solarfind", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Workaround for deploy

In [ ]:
!az login --use-device-code

In [ ]:
!az acr login -n ec0f485ba9a348798fe7130134d9d052

In [ ]:
!docker tag solarfind ec0f485ba9a348798fe7130134d9d052.azurecr.io/solarfind:latest

In [ ]:
!docker push ec0f485ba9a348798fe7130134d9d052.azurecr.io/solarfind:latest